In [42]:

#this model is non functional

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {}'.format(device))
torch.device(device)

Using cuda


device(type='cuda')

In [44]:
#training variables

imgTransformSize = 224
#range of degrees +- to rotate
imgTransformRngRot = 5

epochs = 25

modelLearnRate = 0.01
modelMomentum= 0.5
modelWeightDecay= 0.003

train_dataset_path = './datasets/FoodTrain1'
valid_dataset_path = './datasets/FoodValidate1'

In [45]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(scale=(0.6, 1.0), size=(imgTransformSize,imgTransformSize)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(imgTransformRngRot),
    transforms.ToTensor()
])

valid_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=(imgTransformSize,imgTransformSize)),
    transforms.ToTensor()
])

In [46]:
train_dataset = torchvision.datasets.ImageFolder(root = train_dataset_path, transform = train_transforms)
valid_dataset = torchvision.datasets.ImageFolder(root = valid_dataset_path, transform = valid_transforms)

In [47]:
def show_transformed_images(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size = 6, shuffle=True)
    batch = next(iter(loader))
    images, labels = batch
    
    grid = torchvision.utils.make_grid(images, nrow=3)
    plt.figure(figsize=(11,11))
    plt.imshow(np.transpose(grid,(1,2,0)))
    print('labels: ', labels)

#show_transformed_images(train_dataset)

In [48]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = 32, shuffle = False)

train_losses, valid_losses = [], []
train_accs, valid_accs = [], []


In [49]:
#model init

import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

vgg16_model = models.vgg16(weights = models.VGG16_Weights.IMAGENET1K_V1)
num_object_categories = 40 
#num_ftrs = vgg16_model.classifier[0].in_features
#vgg16_model.classifier[0] = nn.Linear(num_ftrs, num_object_categories)
vgg16_model = vgg16_model.to(device)


#The model we're actually using
usedModel = vgg16_model

from datetime import datetime
now = datetime.now()
checkpointName = now.strftime("%y%m%d%H%M")

checkpointName = 'vgg16_'+checkpointName

In [50]:
loss_fn = nn.CrossEntropyLoss()

optimiser = optim.SGD(usedModel.parameters(), lr=modelLearnRate, momentum=modelMomentum, weight_decay=modelWeightDecay)

In [51]:
def save_checkpoint(model, epoch, optimiser, best_acc):
    state = {
        'epoch': epoch+1,
        'model': model.state_dict(),
        'best_accuracy': best_acc,
        'optimiser' : optimiser.state_dict(),
        'comments':'learning rate: {:.2f} momentum: {:.2f} Weight Decay: {:.5f}'.format(modelLearnRate, modelMomentum, modelWeightDecay)
    }

    torch.save(state, checkpointName+'.pth.tar')

In [52]:
def train_network(model, train_loader, valid_loader, criterion, optimiser, n_epochs):
    
    best_acc = 0
    
    for epoch in range(n_epochs):
        #print("Epoch number %d (epoch + 1)")
        model.train()
        epoch_loss, epoch_accuracy = 0, 0
        epoch_valid_accuracy, epoch_valid_loss = 0, 0
        totalImg = 0
        running_total = 0

        for data in train_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            totalImg += labels.size(0)

            optimiser.zero_grad()

            outputs = model(images)

            loss = criterion(outputs, labels)
            loss.backward()

            optimiser.step()

            acc = ((outputs.argmax(dim=1) == labels).float().mean())

            running_total += (outputs.argmax(dim=1) == labels).sum().item()

            epoch_accuracy += acc/len(train_loader)
            epoch_loss += loss/len(train_loader)

        print('Epoch: {}, train accuracy: {:.2f}%, train loss: {:.4f}'.format(epoch+1, epoch_accuracy*100, epoch_loss), end=" | ")
        train_losses.append(epoch_loss.item())
        train_accs.append(epoch_accuracy.item())

        #print("     -training set got %d out of %d images (%.3f%%)" % (running_total, totalImg, epoch_accuracy*100))
        model.eval()

        with torch.no_grad():

            for data in valid_loader:
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                valid_output = model(images)
                valid_loss = loss_fn(valid_output, labels)

                acc = ((valid_output.argmax(dim=1) == labels).float().mean())
                epoch_valid_accuracy += acc/len(valid_loader)
                epoch_valid_loss += valid_loss/len(valid_loader) 
                
        print('Epoch: {}, validation accuracy: {:.2f}%, valid loss: {:.4f}'.format(epoch+1, epoch_valid_accuracy*100, epoch_valid_loss))
        valid_losses.append(epoch_valid_loss.item())
        valid_losses.append(epoch_valid_accuracy.item())

        if best_acc <= epoch_valid_accuracy:
            best_acc = epoch_valid_accuracy
            save_checkpoint(model, epoch, optimiser, best_acc)

    print("training complete")

    return model


In [53]:
model = train_network(usedModel, train_loader, valid_loader, loss_fn, optimiser, epochs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x40 and 4096x4096)

In [ ]:
checkpoint = torch.load(checkpointName+'.pth.tar')

savedModel = models.models.vgg16()

num_ftrs = savedModel.fc.in_features
savedModel.fc = nn.Linear(num_ftrs, num_object_categories)
savedModel.load_state_dict(checkpoint['model'])

torch.save(savedModel, checkpointName+'.pth')